In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.model_selection import cross_validate

import statsmodels.api as sm 
import statsmodels.formula.api as smf

import math

In [2]:
cs_data = pd.read_csv('redoing_cs_data.csv')
cs_back_data = pd.read_csv('cs_back_data.csv')

In [3]:
cs_data.head()

,sent_type,university,sent_id,aligned_to,original_sentence,eng_translation,zh_translation,cs_word_id,first_cs_word_form,first_cs_word_translation,...,surprisal_values,average_surprisal,bilingual_corpus_frequency_negative_log_first_cs_word,bilingual_corpus_frequency_negative_log_first_cs_word_trans,length_first_cs_word_form,dependency_distance,if_it_is_root,if_previous_word_is_punctuation,surprisal_of_previous_word,surprisal_first_cs_word_trans
0,code-switch,CMU,2,CMU_296,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,Rich neighborhood is very safe (it belongs to ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,...,6.100998 1.320391 0.7630151 4.4722 3.20577 2.7...,3.144718,11.319098,10.402807,4,1,0,NaN,NaN,6.100998
1,code-switch,CMU,4,CMU_3006,lease 从 八月 开始 ， 具体 日期 可以 商量 。,Lease will begin in August and the date is neg...,租约 从 八月 开始 ， 具体 日期 可以 商量 。,1,lease,租约,...,6.527233 3.646975 4.193077 1.173965 0.5694761 ...,3.155464,7.348806,10.066335,5,8,0,NaN,NaN,6.527233
2,code-switch,CMU,5,CMU_70,house 一共 有 两 层 + 地下室 。,House has two floors plus basement.,房子 一共 有 两 层 + 地下室 。,1,house,房子,...,5.512905 5.228868 0.5955775 1.38895 2.136289 4...,3.268832,6.694125,7.084991,5,2,0,NaN,NaN,5.512905
3,code-switch,CMU,6,CMU_6777,一共 有 两 个 可以 洗澡 的 bathroom 。,There are two bathroom for bathing.,一共 有 两 个 可以 洗澡 的 浴室 。,8,bathroom,浴室,...,4.509301 0.2903937 1.38895 0.4321387 3.465795 ...,2.642274,8.611048,9.709660,8,6,0,0.0,0.895995,5.210801
4,code-switch,CMU,9,CMU_4373,整个 house 家具 齐全 。,The whole house is fully furnished.,整个 房子 家具 齐全 。,2,house,房子,...,3.732348 4.488641 5.760885 6.166569 0.8237057,4.194430,6.694125,7.084991,5,1,0,0.0,3.732348,4.488641


In [4]:
cs_back_data.head()

,Unnamed: 0,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh
0,0,8.291760,AD,2,7.232752
1,1,6.377483,P,1,5.668829
2,2,12.623917,AD,2,6.904053
3,3,11.908693,NN,2,7.180783
4,4,4.681214,CC,1,3.416878


In [5]:
cs_data_all = cs_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh']]

In [6]:
cs_back_data = cs_back_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh']]

In [7]:
code_switched = [1]* len(cs_data_all) #coding original code-switch as 1, code-switch back as 0

In [8]:
cs_data_all['code_switched'] = code_switched

/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
cs_data_all.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
0,46.615353,NN,2,7.351441,1
1,41.578729,NN,2,7.351441,1
2,37.403903,NN,2,7.039197,1
3,40.402814,NN,2,7.217810,1
4,37.403903,NN,2,6.872654,1


In [10]:
code_switched = [0]* len(cs_back_data) #coding original code-switch as 1, code-switch back as 0

In [11]:
cs_back_data['code_switched'] = code_switched

In [12]:
cs_back_data.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
0,8.291760,AD,2,7.232752,0
1,6.377483,P,1,5.668829,0
2,12.623917,AD,2,6.904053,0
3,11.908693,NN,2,7.180783,0
4,4.681214,CC,1,3.416878,0


In [13]:
frames = [cs_data_all, cs_back_data]
all_data = pd.concat(frames)
all_data.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
0,46.615353,NN,2,7.351441,1
1,41.578729,NN,2,7.351441,1
2,37.403903,NN,2,7.039197,1
3,40.402814,NN,2,7.217810,1
4,37.403903,NN,2,6.872654,1


In [14]:
all_data.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
1123,13.585461,NN,2,7.167052,0
1124,15.820484,NN,2,7.167052,0
1125,13.380050,NN,2,7.351441,0
1126,7.292286,NN,2,6.886385,0
1127,5.134998,VE,1,5.239348,0


In [15]:
as_ints = []
for pos in all_data['cs_word_pos_zh']:
    if pos == 'NN' or pos == 'NR' or pos == 'NT':
        as_ints.append(0)
    elif pos == 'VE' or pos == 'VV':
        as_ints.append(1)
    else:
        as_ints.append(2)

In [16]:
all_data['cs_word_pos_zh'] = as_ints

In [17]:
all_data.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
1123,13.585461,0,2,7.167052,0
1124,15.820484,0,2,7.167052,0
1125,13.380050,0,2,7.351441,0
1126,7.292286,0,2,6.886385,0
1127,5.134998,1,1,5.239348,0


In [18]:
all_data_final = all_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh']]

In [19]:
all_data_final.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh
1123,13.585461,0,2,7.167052
1124,15.820484,0,2,7.167052
1125,13.380050,0,2,7.351441
1126,7.292286,0,2,6.886385
1127,5.134998,1,1,5.239348


In [20]:
cs_word_frequency = np.array(all_data_final['word_freq_zh'])
cs_word_length = np.array(all_data_final['cs_word_length_zh'])
ngram_surp_val = np.array(all_data_final['ngram_surp_zh'])

cs_word_frequency_new = (cs_word_frequency-np.mean(cs_word_frequency))*0.5/np.std(cs_word_frequency)
cs_word_length_new = (cs_word_length-np.mean(cs_word_length))*0.5/np.std(cs_word_length)
ngram_surp_new = (ngram_surp_val-np.mean(ngram_surp_val))*0.5/np.std(ngram_surp_val)

all_data_final['word_freq_zh'] = cs_word_frequency_new
all_data_final['cs_word_length_zh'] = cs_word_length_new
all_data_final['ngram_surp_zh'] = ngram_surp_new

/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [21]:
all_data_final.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh
1123,-0.386793,0,0.196380,0.225634
1124,-0.313383,0,0.196380,0.225634
1125,-0.393540,0,0.196380,0.320739
1126,-0.593495,0,0.196380,0.080869
1127,-0.664352,1,-0.558972,-0.768651


In [22]:
X = all_data_final
y = all_data['code_switched']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [23]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       362
           1       1.00      1.00      1.00       420

    accuracy                           1.00       782
   macro avg       1.00      1.00      1.00       782
weighted avg       1.00      1.00      1.00       782



In [25]:
all_data_final_new = X.copy()
all_data_final_new['code_switched'] = y.copy()

In [26]:
model = smf.ols(formula="code_switched ~ word_freq_zh + C(cs_word_pos_zh) + cs_word_length_zh + ngram_surp_zh",data=all_data_final_new).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          code_switched   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                 1.601e+04
Date:                Mon, 09 May 2022   Prob (F-statistic):               0.00
Time:                        14:43:48   Log-Likelihood:                 2638.1
No. Observations:                2604   AIC:                            -5264.
Df Residuals:                    2598   BIC:                            -5229.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.5606      0.003    199.532      0.000       0.555       0.566
C(cs_word_pos_zh)[T.1]    -0.0211      0.005     -4.160      0.000      -0.031      -0.011
C(cs_word_pos_zh)[T.2]     0.0266      0.005      5.818      0.000       0.018       0.036
word_freq_zh               1.0504      0.004    252.057      0.000       1.042       1.059
cs_word_length_zh         -0.1065      0.004    -26.392      0.000      -0.114      -0.099
ngram_surp_zh             -0.0725      0.004    -17.634      0.000      -0.081      -0.064
==============================================================================
Omnibus:                      642.083   Durbin-Watson:                   1.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2430.476
Skew:                          -1.173   Prob(JB):                         0.00
Kurtosis:                       7.111   Cond. No.                         4.08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
model = smf.ols(formula="code_switched ~ (word_freq_zh + C(cs_word_pos_zh) + cs_word_length_zh + ngram_surp_zh)**2",data=all_data_final_new).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          code_switched   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     5939.
Date:                Mon, 09 May 2022   Prob (F-statistic):               0.00
Time:                        14:44:02   Log-Likelihood:                 2690.6
No. Observations:                2604   AIC:                            -5351.
Df Residuals:                    2589   BIC:                            -5263.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    0.5727      0.004    142.688      0.000       0.565       0.581
C(cs_word_pos_zh)[T.1]                      -0.0281      0.006     -4.791      0.000      -0.040      -0.017
C(cs_word_pos_zh)[T.2]                       0.0205      0.005      3.943      0.000       0.010       0.031
word_freq_zh                                 1.0783      0.007    154.681      0.000       1.065       1.092
word_freq_zh:C(cs_word_pos_zh)[T.1]         -0.0509      0.011     -4.483      0.000      -0.073      -0.029
word_freq_zh:C(cs_word_pos_zh)[T.2]         -0.0393      0.010     -3.830      0.000      -0.059      -0.019
cs_word_length_zh                           -0.0951      0.008    -11.646      0.000      -0.111      -0.079
C(cs_word_pos_zh)[T.1]:cs_word_length_zh    -0.0166      0.015     -1.132      0.258      -0.045       0.012
C(cs_word_pos_zh)[T.2]:cs_word_length_zh    -0.0193      0.010     -1.859      0.063      -0.040       0.001
ngram_surp_zh                               -0.1500      0.013    -11.182      0.000      -0.176      -0.124
C(cs_word_pos_zh)[T.1]:ngram_surp_zh         0.0465      0.019      2.492      0.013       0.010       0.083
C(cs_word_pos_zh)[T.2]:ngram_surp_zh         0.0795      0.017      4.748      0.000       0.047       0.112
word_freq_zh:cs_word_length_zh              -0.0663      0.012     -5.609      0.000      -0.089      -0.043
word_freq_zh:ngram_surp_zh                  -0.0300      0.012     -2.501      0.012      -0.054      -0.006
cs_word_length_zh:ngram_surp_zh              0.0251      0.010      2.573      0.010       0.006       0.044
==============================================================================
Omnibus:                      704.248   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3153.023
Skew:                          -1.234   Prob(JB):                         0.00
Kurtosis:                       7.793   Cond. No.                         18.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# file = open('code_switched_sentences_cs_back_eng.txt', 'r')
# lines = []
# for line in file.readlines(): 
#     lines.append(line[:-1])

In [19]:
# lengths = []
# for word in lines:
#     lengths.append(len(word))

In [20]:
# import spacy
# from nltk import tokenize
# nlp = spacy.load("en_core_web_sm")

In [21]:
# file = open('code_switched_sentences_cs_back_eng.txt',"r+")
# p = file.read()
# tokenized = p.splitlines()

In [22]:
# print(len(tokenized))

1128


In [23]:
# tags = []
# #words = []
# for word in tokenized:
#     #words.append(word)
#     doc = nlp(word)
#     for token in doc:
#         tags.append(token.tag_)
#         break
        

In [24]:
# print(len(tags))

1128


In [25]:
# import urllib
# import requests

# encoded_query = urllib.parse.quote('testing')
# params = {'corpus': 'eng-us', 'query': encoded_query, 'topk': 3}
# params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

# response = requests.get('https://api.phrasefinder.io/search?' + params)

# assert response.status_code == 200

# print(response.json())

{'phrases': [{'tks': [{'tt': 'testing', 'tg': 0}], 'mc': 11744435, 'vc': 1352477, 'fy': 1500, 'ly': 2009, 'id': 1103806597843, 'sc': 0.8262176938264886}, {'tks': [{'tt': 'Testing', 'tg': 0}], 'mc': 2243991, 'vc': 428730, 'fy': 1586, 'ly': 2009, 'id': 1103806605987, 'sc': 0.15786413471464536}, {'tks': [{'tt': 'TESTING', 'tg': 0}], 'mc': 226272, 'vc': 82093, 'fy': 1643, 'ly': 2009, 'id': 1103806644785, 'sc': 0.015918171458866027}]}


In [26]:
# dict(response.json())['phrases'][0]['mc']

11744435

In [31]:
# match_counts = []

# for word in lines:
#     encoded_query = urllib.parse.quote(str(word))
#     if word == '?':
#         encoded_query = urllib.parse.quote(str('\?'))
#     params = {'corpus': 'eng-us', 'query': encoded_query, 'topk': 3}
#     params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

#     response = requests.get('https://api.phrasefinder.io/search?' + params)

#     assert response.status_code == 200

#     try:
#         match_counts.append(dict(response.json())['phrases'][0]['mc'])
#     except:
#         match_counts.append(1)

In [32]:
# sum = 356033418959

In [33]:
# import numpy as np

In [34]:
# frequencies = [-np.log(num/sum) for num in match_counts]

In [35]:
# len(frequencies) == len(match_counts)

True

In [40]:
# with open('cs_back_word_eng_lengths.txt', 'w') as f:
#     for line in lengths:
#         f.write(str(line))
#         f.write('\n')

In [41]:
# with open('cs_back_word_eng_frequencies.txt', 'w') as f:
#     for line in frequencies:
#         f.write(str(line))
#         f.write('\n')

In [42]:
# with open('cs_back_word_eng_POS.txt', 'w') as f:
#     for line in tags:
#         f.write(str(line))
#         f.write('\n')

In [40]:
cs_data = pd.read_csv('redoing_cs_data.csv')
cs_back_data = pd.read_csv('full_cs_back_data.csv')

In [41]:
cs_data.head()

,sent_type,university,sent_id,aligned_to,original_sentence,eng_translation,zh_translation,cs_word_id,first_cs_word_form,first_cs_word_translation,...,surprisal_values,average_surprisal,bilingual_corpus_frequency_negative_log_first_cs_word,bilingual_corpus_frequency_negative_log_first_cs_word_trans,length_first_cs_word_form,dependency_distance,if_it_is_root,if_previous_word_is_punctuation,surprisal_of_previous_word,surprisal_first_cs_word_trans
0,code-switch,CMU,2,CMU_296,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,Rich neighborhood is very safe (it belongs to ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,...,6.100998 1.320391 0.7630151 4.4722 3.20577 2.7...,3.144718,11.319098,10.402807,4,1,0,NaN,NaN,6.100998
1,code-switch,CMU,4,CMU_3006,lease 从 八月 开始 ， 具体 日期 可以 商量 。,Lease will begin in August and the date is neg...,租约 从 八月 开始 ， 具体 日期 可以 商量 。,1,lease,租约,...,6.527233 3.646975 4.193077 1.173965 0.5694761 ...,3.155464,7.348806,10.066335,5,8,0,NaN,NaN,6.527233
2,code-switch,CMU,5,CMU_70,house 一共 有 两 层 + 地下室 。,House has two floors plus basement.,房子 一共 有 两 层 + 地下室 。,1,house,房子,...,5.512905 5.228868 0.5955775 1.38895 2.136289 4...,3.268832,6.694125,7.084991,5,2,0,NaN,NaN,5.512905
3,code-switch,CMU,6,CMU_6777,一共 有 两 个 可以 洗澡 的 bathroom 。,There are two bathroom for bathing.,一共 有 两 个 可以 洗澡 的 浴室 。,8,bathroom,浴室,...,4.509301 0.2903937 1.38895 0.4321387 3.465795 ...,2.642274,8.611048,9.709660,8,6,0,0.0,0.895995,5.210801
4,code-switch,CMU,9,CMU_4373,整个 house 家具 齐全 。,The whole house is fully furnished.,整个 房子 家具 齐全 。,2,house,房子,...,3.732348 4.488641 5.760885 6.166569 0.8237057,4.194430,6.694125,7.084991,5,1,0,0.0,3.732348,4.488641


In [42]:
cs_back_data.head()

,Unnamed: 0,sentence_length_zh,cs_back_word,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,cs_back_word_eng,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng
0,0,34,非常,8.291760,AD,2,7.232752,very,7.231283,RB,4,5.175176
1,1,10,从,6.377483,P,1,5.668829,from,5.682590,IN,4,3.971755
2,2,8,一共,12.623917,AD,2,6.904053,a total of,11.721629,DT,10,3.771499
3,4,5,家具,11.908693,NN,2,7.180783,furniture,10.891618,NN,9,6.250285
4,6,8,和,4.681214,CC,1,3.416878,and,3.774670,CC,3,3.295646


In [43]:
cs_data_all = cs_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh', 'word_freq_eng', 'cs_word_pos_eng', 'cs_word_length_eng', 'ngram_surp_eng']]

In [44]:
cs_data_all.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng
0,46.615353,NN,2,7.351441,9.642460,JJ,4,5.984415
1,41.578729,NN,2,7.351441,11.246214,NN,5,6.479947
2,37.403903,NN,2,7.039197,8.302733,NNP,5,5.313124
3,40.402814,NN,2,7.217810,11.585480,NNP,8,6.143123
4,37.403903,NN,2,6.872654,8.302733,NNP,5,3.013755


In [45]:
cs_back_data = cs_back_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh', 'word_freq_eng', 'cs_word_pos_eng', 'cs_word_length_eng', 'ngram_surp_eng']]

In [51]:
cs_back_data.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng
0,8.291760,AD,2,7.310274,7.231283,RB,4,5.175176
1,6.377483,P,1,5.059746,5.682590,IN,4,3.971755
2,12.623917,AD,2,5.992529,11.721629,DT,10,3.771499
3,11.908693,NN,2,7.351441,10.891618,NN,9,6.250285
4,4.681214,CC,1,3.577562,3.774670,CC,3,3.295646


In [52]:
code_switched = [1]* len(cs_data_all) #coding original code-switch as 1, code-switch back as 0

In [53]:
cs_data_all['code_switched'] = code_switched

/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
cs_data_all.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng,code_switched
0,46.615353,NN,2,7.351441,9.642460,JJ,4,5.984415,1
1,41.578729,NN,2,7.351441,11.246214,NN,5,6.479947,1
2,37.403903,NN,2,7.039197,8.302733,NNP,5,5.313124,1
3,40.402814,NN,2,7.235478,11.585480,NNP,8,4.542204,1
4,37.403903,NN,2,6.872654,8.302733,NNP,5,3.267129,1


In [55]:
code_switched = [0]* len(cs_back_data) #coding original code-switch as 1, code-switch back as 0

In [56]:
cs_back_data['code_switched'] = code_switched

In [57]:
cs_back_data.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng,code_switched
0,8.291760,AD,2,7.310274,7.231283,RB,4,5.175176,0
1,6.377483,P,1,5.059746,5.682590,IN,4,3.971755,0
2,12.623917,AD,2,5.992529,11.721629,DT,10,3.771499,0
3,11.908693,NN,2,7.351441,10.891618,NN,9,6.250285,0
4,4.681214,CC,1,3.577562,3.774670,CC,3,3.295646,0


In [58]:
frames = [cs_data_all, cs_back_data]
all_data = pd.concat(frames)
all_data.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng,code_switched
0,46.615353,NN,2,7.351441,9.642460,JJ,4,5.984415,1
1,41.578729,NN,2,7.351441,11.246214,NN,5,6.479947,1
2,37.403903,NN,2,7.039197,8.302733,NNP,5,5.313124,1
3,40.402814,NN,2,7.235478,11.585480,NNP,8,4.542204,1
4,37.403903,NN,2,6.872654,8.302733,NNP,5,3.267129,1


In [59]:
all_data.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng,code_switched
1123,13.585461,NN,2,7.351441,11.171262,NNP,3,6.338905,0
1124,15.820484,NN,2,7.351441,11.831533,DT,6,3.771499,0
1125,13.380050,NN,2,7.351441,13.146891,DT,10,3.469161,0
1126,7.292286,NN,2,6.812994,11.447750,DT,17,3.469161,0
1127,5.134998,VE,1,5.096284,8.897150,EX,9,4.660431,0


In [60]:
as_ints = []
for pos in all_data['cs_word_pos_zh']:
    if pos == 'NN' or pos == 'NR' or pos == 'NT':
        as_ints.append(0)
    elif pos == 'VE' or pos == 'VV':
        as_ints.append(1)
    else:
        as_ints.append(2)

In [61]:
all_data['cs_word_pos_zh'] = as_ints

In [64]:
all_data.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng,code_switched
1123,13.585461,0,2,7.351441,11.171262,NNP,3,6.338905,0
1124,15.820484,0,2,7.351441,11.831533,DT,6,3.771499,0
1125,13.380050,0,2,7.351441,13.146891,DT,10,3.469161,0
1126,7.292286,0,2,6.812994,11.447750,DT,17,3.469161,0
1127,5.134998,1,1,5.096284,8.897150,EX,9,4.660431,0


In [66]:
as_ints_eng = []
for pos in all_data['cs_word_pos_eng']:
    if pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS':
        as_ints_eng.append(0)
    elif 'V' in pos:
        as_ints_eng.append(1)
    else:
        as_ints_eng.append(2)

In [69]:
all_data['cs_word_pos_eng'] = as_ints_eng

In [70]:
all_data.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng,code_switched
1123,13.585461,0,2,7.351441,11.171262,0,3,6.338905,0
1124,15.820484,0,2,7.351441,11.831533,2,6,3.771499,0
1125,13.380050,0,2,7.351441,13.146891,2,10,3.469161,0
1126,7.292286,0,2,6.812994,11.447750,2,17,3.469161,0
1127,5.134998,1,1,5.096284,8.897150,2,9,4.660431,0


In [71]:
all_data_final = all_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh', 'word_freq_eng', 'cs_word_pos_eng', 'cs_word_length_eng', 'ngram_surp_eng']]

In [72]:
all_data_final.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng
1123,13.585461,0,2,7.351441,11.171262,0,3,6.338905
1124,15.820484,0,2,7.351441,11.831533,2,6,3.771499
1125,13.380050,0,2,7.351441,13.146891,2,10,3.469161
1126,7.292286,0,2,6.812994,11.447750,2,17,3.469161
1127,5.134998,1,1,5.096284,8.897150,2,9,4.660431


In [73]:
cs_word_frequency = np.array(all_data_final['word_freq_zh'])
cs_word_length = np.array(all_data_final['cs_word_length_zh'])
ngram_surp_val = np.array(all_data_final['ngram_surp_zh'])
cs_word_frequency_eng = np.array(all_data_final['word_freq_eng'])
cs_word_length_eng = np.array(all_data_final['cs_word_length_eng'])
ngram_surp_val_eng = np.array(all_data_final['ngram_surp_eng'])

cs_word_frequency_new = (cs_word_frequency-np.mean(cs_word_frequency))*0.5/np.std(cs_word_frequency)
cs_word_length_new = (cs_word_length-np.mean(cs_word_length))*0.5/np.std(cs_word_length)
ngram_surp_new = (ngram_surp_val-np.mean(ngram_surp_val))*0.5/np.std(ngram_surp_val)
cs_word_frequency_new_eng = (cs_word_frequency_eng-np.mean(cs_word_frequency_eng))*0.5/np.std(cs_word_frequency_eng)
cs_word_length_new_eng = (cs_word_length_eng-np.mean(cs_word_length_eng))*0.5/np.std(cs_word_length_eng)
ngram_surp_new_eng = (ngram_surp_val_eng-np.mean(ngram_surp_val_eng))*0.5/np.std(ngram_surp_val_eng)


all_data_final['word_freq_zh'] = cs_word_frequency_new
all_data_final['cs_word_length_zh'] = cs_word_length_new
all_data_final['ngram_surp_zh'] = ngram_surp_new
all_data_final['word_freq_eng'] = cs_word_frequency_new_eng
all_data_final['cs_word_length_eng'] = cs_word_length_new_eng
all_data_final['ngram_surp_eng'] = ngram_surp_new_eng

/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [74]:
all_data_final.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,word_freq_eng,cs_word_pos_eng,cs_word_length_eng,ngram_surp_eng
1123,-0.386793,0,0.196380,0.293778,0.297023,0,-0.444011,0.566476
1124,-0.313383,0,0.196380,0.293778,0.409921,2,0.084097,-0.306122
1125,-0.393540,0,0.196380,0.293778,0.634831,2,0.788240,-0.408879
1126,-0.593495,0,0.196380,0.020700,0.344299,2,2.020492,-0.408879
1127,-0.664352,1,-0.558972,-0.849945,-0.091820,2,0.612205,-0.003996


In [75]:
X = all_data_final
y = all_data['code_switched']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [76]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [77]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       362
           1       1.00      1.00      1.00       420

    accuracy                           1.00       782
   macro avg       1.00      1.00      1.00       782
weighted avg       1.00      1.00      1.00       782



In [78]:
all_data_final_new = X.copy()
all_data_final_new['code_switched'] = y.copy()

In [80]:
model = smf.ols(formula="code_switched ~ word_freq_zh + C(cs_word_pos_zh) + cs_word_length_zh + ngram_surp_zh + word_freq_eng + C(cs_word_pos_eng) + cs_word_length_eng + ngram_surp_eng + word_freq_zh*word_freq_eng",data=all_data_final_new).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          code_switched   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     7692.
Date:                Thu, 28 Apr 2022   Prob (F-statistic):               0.00
Time:                        15:51:31   Log-Likelihood:                 2711.2
No. Observations:                2604   AIC:                            -5398.
Df Residuals:                    2592   BIC:                            -5328.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.5734      0.003    173.649      0.000       0.567       0.580
C(cs_word_pos_zh)[T.1]        -0.0121      0.005     -2.351      0.019      -0.022      -0.002
C(cs_word_pos_zh)[T.2]         0.0310      0.005      6.128      0.000       0.021       0.041
C(cs_word_pos_eng)[T.1]       -0.0214      0.006     -3.290      0.001      -0.034      -0.009
C(cs_word_pos_eng)[T.2]       -0.0332      0.005     -6.311      0.000      -0.044      -0.023
word_freq_zh                   1.0461      0.005    209.218      0.000       1.036       1.056
cs_word_length_zh             -0.0971      0.004    -24.045      0.000      -0.105      -0.089
ngram_surp_zh                 -0.0622      0.005    -13.585      0.000      -0.071      -0.053
word_freq_eng                 -0.0447      0.005     -8.214      0.000      -0.055      -0.034
cs_word_length_eng            -0.0039      0.004     -0.864      0.388      -0.013       0.005
ngram_surp_eng                 0.0010      0.004      0.271      0.786      -0.006       0.009
word_freq_zh:word_freq_eng    -0.0046      0.009     -0.529      0.597      -0.021       0.012
==============================================================================
Omnibus:                      536.577   Durbin-Watson:                   1.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1783.361
Skew:                          -1.018   Prob(JB):                         0.00
Kurtosis:                       6.506   Cond. No.                         6.65
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""